# Import libraries

In [10]:
%load_ext autoreload
%autoreload 2

import tqdm, sys, os, time
import pandas as pd
import scipy as sp

import scanpy as sc
import anndata as ad
import muon

## Load datasets

In [2]:
ad_scrnaseq_A = muon.ad.read_loom("data/processed/scRNAseq_10x_v3_AIBS.loom")
ad_scrnaseq_A

/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/work/03637/nc

AnnData object with n_obs × n_vars = 176584 × 31053
    obs: 'Amp_Date', 'Amp_Name', 'Amp_PCR_cyles', 'Cell_Capture', 'Donor', 'Gender', 'Lib_Cells', 'Lib_Date', 'Lib_Name', 'Lib_PCR_cycles', 'Lib_PassFail', 'Lib_type', 'Live_Cells', 'Live_percent', 'Mean_Reads_perCell', 'Median_Genes_perCell', 'Median_UMI_perCell', 'Region', 'Replicate_Lib', 'Saturation', 'Seq_batch', 'Total_Cells', 'aggr_num', 'barcode', 'class_label', 'cluster_color', 'cluster_id', 'cluster_label', 'doublet.score', 'exp_component_name', 'gene.counts', 'library_id', 'mapped_reads', 'method', 'nonconf_mapped_reads', 'size', 'subclass_label', 'total.reads', 'tube_barcode', 'umi.counts', 'unmapped_reads'
    var: 'gene_id', 'gene_name', 'var_names'
    obsm: 'X_pca'
    varm: 'PCs'

In [9]:
ad_snrnaseq_B = muon.ad.read_loom("data/processed/snRNAseq_10x_v3_Broad.loom")
ad_snrnaseq_B

/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 215823 × 31053
    obs: 'Allen.class_label', 'Allen.cluster_color', 'Allen.cluster_id', 'Allen.cluster_label', 'Allen.subclass_label', 'Broad.QC.Mito', 'Broad.QC.doublet', 'Broad.passQC', 'Comb.QC', 'MALE', 'QC', 'cl', 'class_label', 'cluster', 'cluster_color', 'cluster_id', 'cluster_label', 'comb.QC', 'dataset', 'gene.counts', 'nGene', 'nUMI', 'sample_name', 'size', 'subclass_label', 'umi.counts'
    var: 'gene_name'
    obsm: 'X_pca'
    varm: 'PCs'

In [11]:
mdata = muon.MuData({"scrna_A": ad_scrnaseq_A.copy(), "snrna_B": ad_snrnaseq_B.copy()}, axis=0)
# Now rna is count-based and log_norm_rna is log-normalized
# mdata.mod["rna"].X = mdata.mod["rna"].layers["counts"]
# del mdata.mod["rna"].raw
# del mdata.mod["rna"].layers["counts"]
# del mdata.mod["log_norm_rna"].layers["counts"]
mdata

/work/03637/nct7456/ls6/micromamba/lib/python3.9/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


MuData object with n_obs × n_vars = 215823 × 62106
  2 modalities
    scrna_A:	176584 x 31053
      obs:	'Amp_Date', 'Amp_Name', 'Amp_PCR_cyles', 'Cell_Capture', 'Donor', 'Gender', 'Lib_Cells', 'Lib_Date', 'Lib_Name', 'Lib_PCR_cycles', 'Lib_PassFail', 'Lib_type', 'Live_Cells', 'Live_percent', 'Mean_Reads_perCell', 'Median_Genes_perCell', 'Median_UMI_perCell', 'Region', 'Replicate_Lib', 'Saturation', 'Seq_batch', 'Total_Cells', 'aggr_num', 'barcode', 'class_label', 'cluster_color', 'cluster_id', 'cluster_label', 'doublet.score', 'exp_component_name', 'gene.counts', 'library_id', 'mapped_reads', 'method', 'nonconf_mapped_reads', 'size', 'subclass_label', 'total.reads', 'tube_barcode', 'umi.counts', 'unmapped_reads'
      var:	'gene_id', 'gene_name', 'var_names'
      obsm:	'X_pca'
      varm:	'PCs'
    snrna_B:	215823 x 31053
      obs:	'Allen.class_label', 'Allen.cluster_color', 'Allen.cluster_id', 'Allen.cluster_label', 'Allen.subclass_label', 'Broad.QC.Mito', 'Broad.QC.doublet', 'Broad.passQC', 'Comb.QC', 'MALE', 'QC', 'cl', 'class_label', 'cluster', 'cluster_color', 'cluster_id', 'cluster_label', 'comb.QC', 'dataset', 'gene.counts', 'nGene', 'nUMI', 'sample_name', 'size', 'subclass_label', 'umi.counts'
      var:	'gene_name'
      obsm:	'X_pca'
      varm:	'PCs'